<a href="https://colab.research.google.com/github/Nicompag/ISYS5002_portfolio/blob/main/ISYS5002_Assignment_20201867_Nico_Engler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Modules

In [60]:
#!pip install yfinance
#import yfinance as yfin
#yfin.pdr_override()
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


#1. Get Input

In [61]:
ticker = input('''Please enter the company's stock market ticker symbol: ''')

# Input validation
#if ticker.isdigit() == True:
#  print('Please enter a valid stock market ticker symbol!')



Please enter the company's stock market ticker symbol: MSFT


#2. Get Data

In [62]:
# Establish Session
url = 'https://au.finance.yahoo.com/'
wd.get(url)

In [63]:
# Work out URL pattern
template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'

url = template.format(ticker1 = ticker, ticker2 = ticker)
wd.get(url)

In [64]:
def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)

  return url

In [65]:
soup = BeautifulSoup(wd.page_source, 'html.parser')

#3. Extract Collection

In [66]:
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
table[4].td.text

'Brett  Iversen'

In [67]:
def extract_name(item):
  name = item.td.text
  return name

In [68]:
item = table[0]
salary = item.find('td', 'Ta(end)')
salary = salary.text

salary[-1]

'M'

In [81]:
def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text 
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000
  except ValueError:
    salary = 'N/A'
  return salary

In [83]:
extract_salary(table[6])

'N/A'

In [84]:
def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

In [124]:
def calculate_tax(income):
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 'N/A'
  return tax_payable

In [125]:
def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

In [126]:
for i in table:
  print(extract_name(i))

Mr. Satya  Nadella
Mr. Bradford L. Smith LCA
Ms. Amy E. Hood
Ms. Alice L. Jolla
Brett  Iversen
Mr. Frank X. Shaw
Mr. Christopher C. Capossela
Mr. Keith Ranger Dolliver Esq.
Ms. Kathleen T. Hogan
Mr. Ali Mohamed El Faramawy


In [127]:
records = []
for i in table:
  records.append(extract_record(i))

In [128]:
df = pd.DataFrame.from_records(records)
df

,Name,Title,Salary,Expected income tax
0,Mr. Satya Nadella,Chairman & CEO,1.36e+07,6.09067e+06
1,Mr. Bradford L. Smith LCA,"Pres, Chief Legal Officer & Vice Chairman",4.27e+06,1.89217e+06
2,Ms. Amy E. Hood,Exec. VP & CFO,4.83e+06,2.14417e+06
3,Ms. Alice L. Jolla,Corp. VP of Fin. & Admin. and Chief Accounting...,N/A,N/A
4,Brett Iversen,Gen. Mang. of Investor Relations,N/A,N/A
5,Mr. Frank X. Shaw,Corp. VP for Corp. Communications,N/A,N/A
6,Mr. Christopher C. Capossela,Exec. VP of Marketing & Consumer Bus. and Chie...,N/A,N/A
7,Mr. Keith Ranger Dolliver Esq.,"VP, Deputy Gen. Counsel of Corp., External & L...",N/A,N/A
8,Ms. Kathleen T. Hogan,Exec. VP & Chief HR Officer,N/A,N/A
9,Mr. Ali Mohamed El Faramawy,Head of The Emerging Markets Digital Transform...,N/A,N/A


#4. Putting it together

In [24]:
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)
  return url

def extract_name(item):
  name = item.td.text
  return name

def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text 
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000
  except ValueError:
    salary = 'N/A'
  return salary

def calculate_tax(income):
  try:
    if income <= 18200:
      tax_payable = 0
    elif 18200 < income <= 45000:
      tax_payable = (income - 18200) * 0.19
    elif 45000 < income <= 120000:
      tax_payable = 5092 + 0.325 * (income - 45000)
    elif 120000 < income <= 180000:
      tax_payable = 29467 + 0.37 * (income - 120000)
    elif income > 180000:
      tax_payable = 51667 + 0.45 * (income - 180000)
  except TypeError:
      tax_payable = 'N/A'
  return tax_payable

def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item),
      'Expected income tax': calculate_tax(extract_salary(item))
  }

def ticker_validation(dataframe):
  if dataframe.empty == True:
    print('Please enter a valid ticker.')

In [30]:
ticker = input('''Please enter the company's stock market ticker symbol: ''')
url = get_url(ticker)
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')
records = []
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})

for i in table:
  records.append(extract_record(i))

df = pd.DataFrame.from_records(records)
ticker_validation(df)
df

Please enter the company's stock market ticker symbol: TAH.AX


,Name,Title,Salary,Expected income tax
0,"Mr. David Robert Henry Attenborough BSc, MBA","MD, CEO & Exec. Director",2.94e+06,1.29367e+06
1,Mr. Adam Newman,Chief Financial Officer,1.18e+06,501667
2,Mr. Patrick J. McGlinchey,Chief Legal & Risk Officer and Co-Company Sec.,853000,354517
3,Mr. Adam Rytenskild,Managing Director of Wagering & Media,1.26e+06,537667
4,Ms. Sue van der Merwe,Managing Director of Lotteries & Keno,1.11e+06,470167
5,Ms. Francoise Russo,Chief Information Officer,N/A,N/A
6,Mr. Chris Richardson,Gen. Mang. of Investor Relations & Treasury,N/A,N/A
7,Mr. Nicholas Tzaferis,Gen. Mang. of Corp. Communications,N/A,N/A
8,Ms. Michelle Williams,Chief People Officer,N/A,N/A
9,Mr. Ben Simons,Chief Strategy Officer,N/A,N/A
